In [3]:
import pandas as pd
from collections import defaultdict

file_path = 'Schedule formation (INF 236, INF 393, INF 368) (Responses) - Form Responses 1.csv'
df = pd.read_csv(file_path)

df['Selected Time Slots'] = df['We want to defend in (select all that apply and convenient for you, choose the ones corresponded to the subject)'].apply(lambda x: x.split(','))

teams_by_subject = defaultdict(list)
for index, row in df.iterrows():
    team_ids = row['List your team members IDs in the form id1, id2, ..., id_n (e.g. 1232131, 123213, 213213)']
    subject = row['Subject']
    for time_slot in row['Selected Time Slots']:
        teams_by_subject[subject].append((team_ids, time_slot.strip()))

time_slots_updated = {
    "INF 236": ["12:00-16:00 Wednesday F108", "09:00-11:00 Thursday G205", "16:00-17:00 Thursday F305"],
    "INF 393": ["10:00-12:00 Tuesday F108", "11:00-13:00 Thursday F108", "16:00-17:00 Friday F108"],
    "INF 368": ["12:00-14:00 Wednesday F301", "14:00-16:00 Thursday F204", "13:00-15:00 Friday G112"],
}

def distribute_teams_balanced(teams_by_subject, time_slots):
    participant_log = defaultdict(list)
    schedule = defaultdict(list)
    slot_balancing = defaultdict(int)

    for subject, teams_info in teams_by_subject.items():
        for team_info in teams_info:
            team_ids, preferred_slot = team_info
            preferred_slots = [slot for slot in time_slots[subject] if preferred_slot in slot]
            sorted_slots = sorted(preferred_slots, key=lambda x: slot_balancing[x])
            for slot in sorted_slots:
                if not any(participant in participant_log[slot] for participant in team_ids.split(', ')):
                    schedule[slot].append(f"{subject}: Team {team_ids}")
                    participant_log[slot] += team_ids.split(', ')
                    slot_balancing[slot] += 1
                    break

    return schedule

balanced_schedule = distribute_teams_balanced(teams_by_subject, time_slots_updated)

schedule_str = ""
for slot, teams in sorted(balanced_schedule.items(), key=lambda x: x[0]):
    schedule_str += f"Временной слот: {slot}\n"
    for team in teams:
        team = team.replace('\n', '') 
        schedule_str += f"- {team.strip()}\n"  
    schedule_str += "\n" 

print(schedule_str)


Временной слот: 09:00-11:00 Thursday G205
- INF 236: Team 210103258, 210103170
- INF 236: Team 210103252, 210103191
- INF 236: Team 210103371, 210103305
- INF 236: Team 210109038210103213
- INF 236: Team 210103461, 210103214, 210103274
- INF 236: Team 210103167, 210103425, 210103116
- INF 236: Team 210107068, 210107039, 210103239
- INF 236: Team 210103084, 210103421, 210103022
- INF 236: Team 210103065, 210103272
- INF 236: Team 210103182, 210103345

Временной слот: 10:00-12:00 Tuesday F108
- INF 393: Team 210103007, 210103243
- INF 393: Team 210107039, 210107053, 210103318
- INF 393: Team 210107070, 210103447, 210103129
- INF 393: Team 210103442, 210103373, 210103300
- INF 393: Team 210107108, 210107100
- INF 393: Team 210103410, 210103246

Временной слот: 11:00-13:00 Thursday F108
- INF 393: Team 210103381, 210103382
- INF 393: Team 210103007, 210103243
- INF 393: Team 210107070, 210103447, 210103129
- INF 393: Team id1 : 210103381id2 : 210103382id3 : 210103459
- INF 393: Team 210103